### Callback API
- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면, 모든 훈련 시간이 낭비될 수 있고,  
  과적합을 방지하기 위해 훈련을 중간에 중지해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 아무런 제어를 하지 못하게 되고,  
  신경망 훈련을 완료하는 데에는 몇 시간 또는 며칠이 걸릴 수 있기 때문에 모델을 모니터링하고 제어할 수 있는 기능이 필요하다.
- 훈련 시(fit()) Callback API를 등록시키면 반복 내에서 특정 이벤트 발생마다 등록된 callback이 호출되어 수행된다.

**ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weight_only=False, mode='auto')**
- 특정 조건에 따라서 모델 또는 가중치를 파일로 저장한다.
- filepath: "weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5"와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- save_best_only: 가장 좋은 성능을 나타내는 모델을 저장할 지에 대한 여부
- save_weights_only: weights만 저장할 지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

<sub>*epoch:03d = 3자리 정수로 자리를 맞추기, 0 붙이면 앞에 공백을 맞춘다 (001, 010, 100 ...) </sub>  
<sub>*확장자는 반드시 weights.h5를 한번에 작성해야 함 (확장자만 작성 불가)</sub>  
<sub>*monitor와 mode는 같이 가는 개념, monitor에 loss 값을 구해야 하면 mode에는 min, acc면 max, auto도 가능하나 잘 사용하지 않음</sub>

**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_lr=0)**
- 특정 반복동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비율, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

**EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')**
- 특정 반복동안 성능이 개선되지 않을 때, 학습을 조기에 중단한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전에 monitor할 반복 횟수.
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

In [1]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

# 입력 데이터 크기 지정
INPUT_SIZE = 28

# 인공 신경망 생성 함수 선언
def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 전처리 함수 선언
def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

# 원-핫 인코딩 변환 함수 선언
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

# 데이터 세트 분리 함수 선언
def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [3]:
from tensorflow.keras.datasets import fashion_mnist

# tensorflow 라이브러리에 내장되어 있는 패션 데이터 세트 호출 후 데이터 세트 분리
(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

# 데이터 세트 분리 함수를 통해 데이터 세트 분리
(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

# 데이터 형태 확인
print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [4]:
# 현제 디렉토리 확인
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 0457-852B

 C:\study\ai\deep_learning\c_tensorflow 디렉터리

2024-05-28  오후 05:48    <DIR>          .
2024-05-28  오전 11:30    <DIR>          ..
2024-05-28  오전 11:29    <DIR>          .ipynb_checkpoints
2024-05-27  오후 08:38            92,212 a_tensorflow.ipynb
2024-05-28  오후 04:55           166,938 b_keras.ipynb
2024-05-27  오후 10:12           184,667 b_keras_task.ipynb
2024-05-28  오후 09:15    <DIR>          callback_files
2024-05-28  오후 05:48            41,569 c_callback.ipynb
2024-05-28  오전 10:42            34,848 c_callback22.ipynb
2024-05-28  오전 11:29            10,534 c_callback_task.ipynb
2024-05-28  오전 11:29            13,199 c_callback_task22.ipynb
2024-05-27  오후 08:15    <DIR>          images
               7개 파일             543,967 바이트
               5개 디렉터리  1,822,870,302,720 바이트 남음


In [5]:
# 현재 디렉토리 내 특정 폴더 확인
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 0457-852B

 C:\study\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오후 09:15    <DIR>          .
2024-05-28  오후 05:48    <DIR>          ..
               0개 파일                   0 바이트
               2개 디렉터리  1,822,870,302,720 바이트 남음


### ModelCheckPoint

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# weights 저장
mcp_cb = ModelCheckpoint(
    # 경로 지정
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    # 성능 지표 최적화
    mode='min'
)

# 훈련
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.7384 - loss: 0.7523 - val_acc: 0.8438 - val_loss: 0.4434
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8567 - loss: 0.4033 - val_acc: 0.8627 - val_loss: 0.3764
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8715 - loss: 0.3589 - val_acc: 0.8668 - val_loss: 0.3624
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8777 - loss: 0.3351 - val_acc: 0.8697 - val_loss: 0.3578
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8865 - loss: 0.3092 - val_acc: 0.8802 - val_loss: 0.3300
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8929 - loss: 0.2923 - val_acc: 0.8734 - val_loss: 0.3366
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8957 - loss: 0.2827 - val_acc: 0.8693 - val_loss: 0.3524
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8982 - loss: 0.2741 - val_acc: 0.8861 - val_loss: 0.3169
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

In [7]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# 모델 저장
mcp_cb = ModelCheckpoint(
    # 경로 지정
    filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=False,
    # 성능 지표 최적화
    mode='min'
)

# 훈련
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.7419 - loss: 0.7569 - val_acc: 0.8425 - val_loss: 0.4347
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8556 - loss: 0.4052 - val_acc: 0.8543 - val_loss: 0.3916
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8693 - loss: 0.3577 - val_acc: 0.8702 - val_loss: 0.3560
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8797 - loss: 0.3262 - val_acc: 0.8708 - val_loss: 0.3485
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8836 - loss: 0.3140 - val_acc: 0.8648 - val_loss: 0.3542
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8931 - loss: 0.2931 - val_acc: 0.8840 - val_loss: 0.3124
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8949 - loss: 0.2789 - val_acc: 0.8842 - val_loss: 0.3158
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8992 - loss: 0.2714 - val_acc: 0.8810 - val_loss: 0.3230
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - ac

#### TIP!
model을 내보내면 그대로 predict하여 사용하겠다는 의미지만,  
weights(가중치)를 내보내는 이유는 weight부터 시작한다는 의미.  
weight를 받아서 바로 predict도 할 수 있지만, 이건 model로도 가능하기 때문에 바로 prediect 시 model을 내보낸다.  
<sub>*weights부터 시작하면 가중치를 가지고 내 이미지 파일을 다시 훈련하는 느낌</sub>

weight를 가져간다는 것은 층도 동일하게 사용하겠다는 것이고 적용 후 튜닝까지 진행 해야한다.  
model 보다 사용할 수 있는 폭이 더 크다.

In [8]:
# 현재 디렉토리 내 특정 폴더 확인
!dir callback_files

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 0457-852B

 C:\study\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오후 09:16    <DIR>          .
2024-05-28  오후 05:48    <DIR>          ..
2024-05-28  오후 09:16           743,334 model.001-0.4347-0.8063.model.keras
2024-05-28  오후 09:16           743,334 model.002-0.3916-0.8567.model.keras
2024-05-28  오후 09:16           743,334 model.003-0.3560-0.8708.model.keras
2024-05-28  오후 09:16           743,334 model.004-0.3485-0.8796.model.keras
2024-05-28  오후 09:16           743,334 model.005-0.3542-0.8851.model.keras
2024-05-28  오후 09:16           743,334 model.006-0.3124-0.8916.model.keras
2024-05-28  오후 09:16           743,334 model.007-0.3158-0.8956.model.keras
2024-05-28  오후 09:16           743,334 model.008-0.3230-0.8990.model.keras
2024-05-28  오후 09:16           743,334 model.009-0.3136-0.9039.model.keras
2024-05-28  오후 09:16           743,334 model.010-0.3177-0.9057.model.keras
2024-05-28  오후 09:16           743,334 model.011-0.3277-0.90

In [9]:
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - acc: 0.8835 - loss: 0.3604


[0.3568391501903534, 0.8826000094413757]

In [10]:
# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 저장된 가중치 파일을 불러오기
model.load_weights('./callback_files/weights.019-0.3230-0.9262.weights.h5')

# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc: 0.8842 - loss: 0.3654


[0.3636404871940613, 0.8835999965667725]

In [11]:
from tensorflow.keras.models import load_model

# 저장된 모데 파일 불러오기
model = load_model('./callback_files/model.020-0.3290-0.9286.model.keras')
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8841 - loss: 0.3563


[0.3618769645690918, 0.8840000033378601]

### ReduceLROnPlateau

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# 일정 기간 동안 성능이 개선되지 않을 시 학습률 동적으로 감소
rlr_cb = ReduceLROnPlateau(
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 감소할 학습률 지정
    factor=0.1,
    # 성능 미개선 반복 횟수
    patience=2,
    # 성능 지표 최적화
    mode='min'
)

# 훈련
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.7362 - loss: 0.7594 - val_acc: 0.8376 - val_loss: 0.4484 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8533 - loss: 0.4135 - val_acc: 0.8609 - val_loss: 0.3844 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8688 - loss: 0.3563 - val_acc: 0.8694 - val_loss: 0.3604 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8795 - loss: 0.3332 - val_acc: 0.8671 - val_loss: 0.3621 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8853 - loss: 0.3112 - val_acc: 0.8740 - val_loss: 0.3373 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8900 - loss: 0.2962 - val_acc: 0.8793 - val_loss: 0.3199 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8954 - loss: 0.2803 - val_acc: 0.8819 - val_loss: 0.3192 - learning_rate: 0.0010
Epoch 

In [13]:
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.8876 - loss: 0.3218


[0.3224271833896637, 0.8871999979019165]

### EarlyStopping

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

# 인공 신경망 생성 함수를 통해 인공 신경망 생성
model = create_model()
# 모델 컴파일
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# 일정 기간 동안 성능이 개선되지 않을 시 학습 조기 중단
ely_cb = EarlyStopping(
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 성능 미개선 반복 횟수
    patience=3,
    # 성능 지표 최적화
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.7413 - loss: 0.7508 - val_acc: 0.8451 - val_loss: 0.4309
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8544 - loss: 0.4085 - val_acc: 0.8605 - val_loss: 0.3815
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8701 - loss: 0.3596 - val_acc: 0.8684 - val_loss: 0.3615
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8765 - loss: 0.3293 - val_acc: 0.8703 - val_loss: 0.3581
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8869 - loss: 0.3101 - val_acc: 0.8767 - val_loss: 0.3334
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8902 - loss: 0.2960 - val_acc: 0.8775 - val_loss: 0.3323
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8961 - loss: 0.2817 - val_acc: 0.8775 - val_loss: 0.3389
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8989 - loss: 0.2720 - val_acc: 0.8869 - val_loss: 0.3155
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - ac

In [15]:
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.8818 - loss: 0.3512


[0.35141339898109436, 0.881600022315979]

### ALL

In [16]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# weights 저장
mcp_cb = ModelCheckpoint(
    # 경로 지정
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True설정
    save_best_only=False,
    save_weights_only=True,
    # 성능 지표 최적화
    mode='min'
)

# 일정 기간 동안 성능이 개선되지 않을 시 학습률 동적으로 감소
rlr_cb = ReduceLROnPlateau(
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 감소할 학습률 지정
    factor=0.1,
    # 성능 미개선 반복 횟수
    patience=2,
    # 성능 지표 최적화
    mode='min'
)

# 일정 기간 동안 성능이 개선되지 않을 시 학습 조기 중단
ely_cb = EarlyStopping(
    # 모니터링 할 성능지표
    monitor='val_loss',
    # 성능 미개선 반복 횟수
    patience=3,
    # 성능 지표 최적화
    mode='min'
)

# 훈련
history = model.fit(x=train_images, y=train_oh_targets, validation_data=(validation_images, validation_oh_targets), batch_size=64, epochs=20, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - acc: 0.7473 - loss: 0.7404 - val_acc: 0.8506 - val_loss: 0.4224 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8499 - loss: 0.4166 - val_acc: 0.8620 - val_loss: 0.3771 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.8688 - loss: 0.3653 - val_acc: 0.8717 - val_loss: 0.3527 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8819 - loss: 0.3279 - val_acc: 0.8753 - val_loss: 0.3391 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8882 - loss: 0.3076 - val_acc: 0.8755 - val_loss: 0.3454 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8895 - loss: 0.2976 - val_acc: 0.8764 - val_loss: 0.3412 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - acc: 0.9069 - loss: 0.2572 - val_acc: 0.8858 - val_loss: 0.3091 - learning_rate: 1.0000e-04
Ep

In [17]:
# 평가
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.8838 - loss: 0.3238


[0.32640039920806885, 0.8823999762535095]